<center><h1>ARIMA TRAINING</h1><center>

In [12]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

In [ ]:
# ============================================================
#  - Year cap: 2008–2024
#  - Focus genres: Pop, Hip-Hop, Latin, Electronic
#  - Walk-forward: 3y train / 2y test, 1-year step, **all folds** (>=10 where possible)
#  - Manual (p,d,q) tried first each fold, then Grid PDQ; best by RMSE
#  - Final (p,d,q) = lowest average RMSE across folds
#  - Holdout: train ≤ 2022 → predict 2023–2024
#  - 1–5Y forecast with CI
#  - Saves: per-genre WF metrics, per-genre WF candidate results (all models tried),
#           ADF results CSV, ACF/PACF plots, history & history+forecast plots, summary CSV
# ============================================================

import os, numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller, acf, pacf

# -------------------
# CONFIG (edit these)
# -------------------
YEAR_MIN, YEAR_MAX = 2008, 2024
TRAIN_YEARS, TEST_YEARS = 3, 2
THRESH_RMSE, THRESH_MAE = 8.0, 6.0

# Manual (p,d,q) baseline per genre
MANUAL_PDQ = {
    "pop": (1, 2, 1),
    "hip-hop": (1, 1, 1),
    "latin": (1, 1, 1),
    "electronic": (1, 1, 1),
}

# Grid for hybrid tuning (document in your paper)
GRID_P = [0, 1, 2, 3]
GRID_D = [0, 1, 2]
GRID_Q = [0, 1, 2, 3]
GRID_PDQ = [(p, d, q) for p in GRID_P for d in GRID_D for q in GRID_Q]

# Optional: disallow trivial mean model (0,0,0). Keep False to match "pure RMSE selection" in thesis.
EXCLUDE_TRIVIAL = False

# ---------------
# Helper functions
# ---------------
def _rmse(y_true, y_pred): 
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

def _mape(y_true, y_pred):
    y_true, y_pred = np.asarray(y_true, float), np.asarray(y_pred, float)
    return float(np.mean(np.abs((y_true - y_pred) / np.maximum(np.abs(y_true), 1e-8))) * 100)

def _fit_fc(y_train, steps, pdq):
    try:
        m = ARIMA(y_train, order=pdq).fit()
        fc = m.forecast(steps=steps)
        return fc, m
    except Exception:
        return None, None

def _windows(years, tr=3, te=2):
    """Return **all** contiguous windows (no cap)."""
    res, total = [], tr + te
    for i in range(0, len(years) - total + 1):
        a, b = years[i:i+tr], years[i+tr:i+total]
        if all(np.diff(a) == 1) and all(np.diff(b) == 1) and a[-1] + 1 == b[0]:
            res.append((a, b))
    return res

def _ensure_year_series(series):
    if np.issubdtype(series.dtype, np.number):
        return series.astype(int)
    return pd.to_datetime(series, errors="coerce").dt.year.astype("Int64").dropna().astype(int)

def _diagnostics_genre(ts, genre, out_diag_dir):
    """Save ADF results CSV and ACF/PACF plots for the series."""
    os.makedirs(out_diag_dir, exist_ok=True)
    # ADF on level
    adf_stat, adf_p, *_ = adfuller(ts, autolag='AIC')
    diag_df = pd.DataFrame([{"series": "level", "adf_stat": adf_stat, "p_value": adf_p}])

    # ADF on 1st diff
    if len(ts) >= 3:
        ts1 = ts.diff().dropna()
        adf_stat1, adf_p1, *_ = adfuller(ts1, autolag='AIC')
        diag_df.loc[len(diag_df)] = {"series": "diff1", "adf_stat": adf_stat1, "p_value": adf_p1}

    # ADF on 2nd diff (optional)
    if len(ts) >= 4:
        ts2 = ts.diff().dropna().diff().dropna()
        if len(ts2) >= 3:
            adf_stat2, adf_p2, *_ = adfuller(ts2, autolag='AIC')
            diag_df.loc[len(diag_df)] = {"series": "diff2", "adf_stat": adf_stat2, "p_value": adf_p2}

    diag_df.to_csv(os.path.join(out_diag_dir, f"{genre}_adf_results.csv"), index=False)

    # ACF/PACF on level (small lags due to annual data length)
    max_lags = min(8, len(ts) - 1) if len(ts) > 2 else 1
    if max_lags >= 1:
        try:
            ac = acf(ts, nlags=max_lags, fft=False)
            pc = pacf(ts, nlags=max_lags, method='ywunbiased')
            # Plot ACF
            plt.figure()
            plt.stem(range(len(ac)), ac, use_line_collection=True)
            plt.title(f"ACF — {genre.title()} (level)")
            plt.xlabel("Lag"); plt.ylabel("ACF")
            plt.grid(True); plt.tight_layout()
            plt.savefig(os.path.join(out_diag_dir, f"{genre}_acf_level.png")); plt.close()
            # Plot PACF
            plt.figure()
            plt.stem(range(len(pc)), pc, use_line_collection=True)
            plt.title(f"PACF — {genre.title()} (level)")
            plt.xlabel("Lag"); plt.ylabel("PACF")
            plt.grid(True); plt.tight_layout()
            plt.savefig(os.path.join(out_diag_dir, f"{genre}_pacf_level.png")); plt.close()
        except Exception:
            pass

def run_thesis_arima(df, genres, manual_pdq, out_dir="outputs/arima"):
    # Prepare dirs
    os.makedirs(out_dir, exist_ok=True)
    out_diag = os.path.join(out_dir, "diagnostics")
    os.makedirs(out_diag, exist_ok=True)

    # Normalize, cap, filter
    df = df.copy()
    if "playlist_genre" not in df.columns or "track_album_release_date" not in df.columns:
        raise ValueError("Required columns missing: playlist_genre, track_album_release_date")

    df["playlist_genre"] = df["playlist_genre"].str.lower()
    df["year"] = _ensure_year_series(df["track_album_release_date"])
    df = df.dropna(subset=["year"])
    df["year"] = df["year"].astype(int)
    df = df[(df["year"] >= YEAR_MIN) & (df["year"] <= YEAR_MAX)]
    df = df[df["playlist_genre"].isin([g.lower() for g in genres])]
    df = df.sort_values(["playlist_genre", "year"])

    if "median_track_popularity" not in df.columns:
        raise ValueError("Required column missing: median_track_popularity")

    # Log the grid used (for your appendix)
    with open(os.path.join(out_dir, "grid_pdq_used.txt"), "w") as f:
        f.write(f"Manual PDQ: {manual_pdq}\n")
        f.write("Grid PDQ:\n")
        for pdq in GRID_PDQ:
            if EXCLUDE_TRIVIAL and pdq == (0,0,0):
                continue
            f.write(f"{pdq}\n")

    summary_rows = []

    for genre in genres:
        g = df[df["playlist_genre"] == genre].copy()
        if g.empty:
            continue

        ts = g.set_index("year")["median_track_popularity"].astype(float).sort_index()
        years_all = ts.index.tolist()
        if len(ts) < 5:
            print(f"[{genre}] Not enough history — skipping.")
            continue

        # Diagnostics (ADF, ACF/PACF)
        _diagnostics_genre(ts, genre, out_diag)

        # Separate holdout years
        holdout = [y for y in (2023, 2024) if y in years_all]
        years_wf = [y for y in years_all if y not in holdout]

        # Build **all** 3y/2y windows (no cap)
        windows = _windows(years_wf, TRAIN_YEARS, TEST_YEARS)
        if len(windows) < 10:
            print(f"[{genre}] Only {len(windows)} folds available.")
        # Storage
        fold_rows = []
        candidate_rows = []  # all models tried per fold (for transparency)
        pdq_rmse_accum = {}  # (p,d,q): (rmse_sum, count)

        # Walk-forward
        for fold_id, (tr, te) in enumerate(windows, start=1):
            y_tr, y_te = ts.loc[tr], ts.loc[te]

            # Manual first
            best_pdq = manual_pdq.get(genre, (1,1,1))
            if EXCLUDE_TRIVIAL and best_pdq == (0,0,0):
                best_pdq = (1,1,1)

            best_fc, _ = _fit_fc(y_tr, len(y_te), best_pdq)
            best_rmse = np.inf; best_mae = np.inf; best_mape = np.inf
            if best_fc is not None:
                best_rmse = _rmse(y_te.values, best_fc.values)
                best_mae  = float(mean_absolute_error(y_te.values, best_fc.values))
                best_mape = _mape(y_te.values, best_fc.values)
                candidate_rows.append({
                    "genre": genre, "fold": fold_id,
                    "train_start": tr[0], "train_end": tr[-1],
                    "test_start": te[0], "test_end": te[-1],
                    "p": best_pdq[0], "d": best_pdq[1], "q": best_pdq[2],
                    "rmse": best_rmse, "mae": best_mae, "mape": best_mape,
                    "source": "manual"
                })

            # Grid search
            for pdq in GRID_PDQ:
                if EXCLUDE_TRIVIAL and pdq == (0,0,0):
                    continue
                if best_fc is not None and pdq == best_pdq:
                    continue
                fc, _ = _fit_fc(y_tr, len(y_te), pdq)
                if fc is None:
                    continue
                r = _rmse(y_te.values, fc.values)
                a = float(mean_absolute_error(y_te.values, fc.values))
                m = _mape(y_te.values, fc.values)
                candidate_rows.append({
                    "genre": genre, "fold": fold_id,
                    "train_start": tr[0], "train_end": tr[-1],
                    "test_start": te[0], "test_end": te[-1],
                    "p": pdq[0], "d": pdq[1], "q": pdq[2],
                    "rmse": r, "mae": a, "mape": m,
                    "source": "grid"
                })
                if r < best_rmse:
                    best_pdq, best_rmse, best_mae, best_mape = pdq, r, a, m

            # Record fold winner
            fold_rows.append({
                "genre": genre,
                "fold": fold_id,
                "train_start": tr[0], "train_end": tr[-1],
                "test_start": te[0], "test_end": te[-1],
                "p": best_pdq[0], "d": best_pdq[1], "q": best_pdq[2],
                "rmse": best_rmse, "mae": best_mae, "mape": best_mape
            })
            s, c = pdq_rmse_accum.get(best_pdq, (0.0, 0))
            pdq_rmse_accum[best_pdq] = (s + best_rmse, c + 1)

        # Save per-genre fold winners
        wf_df = pd.DataFrame(fold_rows)
        wf_df.to_csv(os.path.join(out_dir, f"{genre}_walkforward_metrics.csv"), index=False)

        # Save per-genre **all candidates** per fold (transparency)
        cand_df = pd.DataFrame(candidate_rows)
        cand_df.to_csv(os.path.join(out_dir, f"{genre}_wf_candidates.csv"), index=False)

        # Final PDQ by lowest average RMSE
        if pdq_rmse_accum:
            final_pdq = min(pdq_rmse_accum.items(), key=lambda kv: kv[1][0] / max(kv[1][1], 1))[0]
        else:
            final_pdq = MANUAL_PDQ.get(genre, (1,1,1))

        # Averages across WF
        avg_rmse = float(wf_df["rmse"].mean()) if not wf_df.empty else np.nan
        avg_mae  = float(wf_df["mae"].mean())  if not wf_df.empty else np.nan
        avg_mape = float(wf_df["mape"].mean()) if not wf_df.empty else np.nan

        # Holdout 2023–2024
        hold_rmse = hold_mae = hold_mape = np.nan
        if set((2023, 2024)).issubset(set(years_all)):
            pre = [y for y in years_all if y <= 2022]
            if len(pre) >= 3:
                y_tr_full, y_te_hold = ts.loc[pre], ts.loc[[2023, 2024]]
                fc_h, _ = _fit_fc(y_tr_full, len(y_te_hold), final_pdq)
                if fc_h is not None:
                    hold_rmse = _rmse(y_te_hold.values, fc_h.values)
                    hold_mae  = float(mean_absolute_error(y_te_hold.values, fc_h.values))
                    hold_mape = _mape(y_te_hold.values, fc_h.values)
                    pd.DataFrame({
                        "year": [2023, 2024],
                        "actual": y_te_hold.values,
                        "forecast": fc_h.values
                    }).to_csv(os.path.join(out_dir, f"{genre}_holdout_2023_2024_vs_forecast.csv"), index=False)

        # Final 1–5Y forecast & plots
        try:
            fitted = ARIMA(ts, order=final_pdq).fit()
            fc_res = fitted.get_forecast(steps=5)
            fc_mean = fc_res.predicted_mean
            fc_ci = fc_res.conf_int()
            last_year = years_all[-1]
            fut_years = list(range(last_year + 1, last_year + 6))

            pd.DataFrame({"year": fut_years, "forecast_popularity": fc_mean.values}).to_csv(
                os.path.join(out_dir, f"{genre}_forecast_1to5y.csv"), index=False
            )

            # History
            plt.figure()
            plt.plot(years_all, ts.values, marker="o")
            plt.title(f"{genre.title()} — Median Popularity (History)")
            plt.xlabel("Year"); plt.ylabel("Median Popularity"); plt.grid(True); plt.tight_layout()
            plt.savefig(os.path.join(out_dir, f"{genre}_history.png")); plt.close()

            # History + Forecast
            plt.figure()
            plt.plot(years_all, ts.values, marker="o", label="History")
            plt.plot(fut_years, fc_mean.values, marker="o", linestyle="--", label="Forecast")
            if isinstance(fc_ci, pd.DataFrame) and fc_ci.shape[0] == 5:
                lower, upper = fc_ci.iloc[:, 0].values, fc_ci.iloc[:, 1].values
                plt.fill_between(fut_years, lower, upper, alpha=0.2, label="95% CI")
            plt.title(f"{genre.title()} — 1–5Y Forecast (ARIMA {final_pdq})")
            plt.xlabel("Year"); plt.ylabel("Median Popularity")
            plt.grid(True); plt.legend(); plt.tight_layout()
            plt.savefig(os.path.join(out_dir, f"{genre}_history_forecast.png")); plt.close()
        except Exception:
            pass

        summary_rows.append({
            "genre": genre,
            "folds": int(len(windows)),
            "avg_rmse_walkforward": avg_rmse,
            "avg_mae_walkforward":  avg_mae,
            "avg_mape_walkforward": avg_mape,
            "final_pdq": str(final_pdq),
            "holdout_rmse_2023_2024": hold_rmse,
            "holdout_mae_2023_2024":  hold_mae,
            "holdout_mape_2023_2024": hold_mape,
            "meets_thresholds_walkforward": bool(
                (avg_rmse <= THRESH_RMSE) and (avg_mae <= THRESH_MAE)
            ) if not (np.isnan(avg_rmse) or np.isnan(avg_mae)) else False
        })

    summary_df = pd.DataFrame(summary_rows)
    summary_df.to_csv(os.path.join(out_dir, "arima_summary_by_genre.csv"), index=False)
    print("Done. Outputs in:", out_dir)
    return summary_df


In [14]:
df = pd.read_csv("05_genres_median_audio_features.csv")
genres = ['pop', 'hip-hop', 'latin', 'electronic']

manual_pdq = {
    'pop': (1, 2, 1),
    'hip-hop': (1, 1, 1),
    'latin': (1, 1, 1),
    'electronic': (1, 1, 1),
}

summary = run_thesis_arima(df, genres, manual_pdq, out_dir="outputs/arima")
summary


c:\Users\jonaz\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
c:\Users\jonaz\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
c:\Users\jonaz\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
c:\Users\jonaz\AppData\Local\Programs\Python\Python313\Lib

[hip-hop] Only 7 folds available.


c:\Users\jonaz\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
c:\Users\jonaz\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
c:\Users\jonaz\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
c:\Users\jonaz\AppData\Local\Programs\Python\Python313\Lib

[latin] Only 6 folds available.


c:\Users\jonaz\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
c:\Users\jonaz\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
c:\Users\jonaz\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
c:\Users\jonaz\AppData\Local\Programs\Python\Python313\Lib

[electronic] Only 9 folds available.


c:\Users\jonaz\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
c:\Users\jonaz\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
c:\Users\jonaz\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
c:\Users\jonaz\AppData\Local\Programs\Python\Python313\Lib

Done. Outputs in: outputs/arima


,genre,folds,avg_rmse_walkforward,avg_mae_walkforward,avg_mape_walkforward,final_pdq,holdout_rmse_2023_2024,holdout_mae_2023_2024,holdout_mape_2023_2024,meets_thresholds_walkforward
0,pop,11,1.267234,1.175323,1.509800,"(2, 1, 3)",3.354100,2.999997,3.561690,True
1,hip-hop,7,0.698477,0.627521,0.874967,"(0, 0, 0)",2.017283,1.583328,2.237760,True
2,latin,6,1.528769,1.406195,1.996189,"(2, 0, 3)",5.441288,4.353111,5.551877,True
3,electronic,9,1.593400,1.516957,2.044234,"(1, 0, 0)",3.571574,3.527381,5.009264,True
